# 🖼️ Multi-Modal Magic: Image & Text Search with Weaviate

Welcome to the frontier of AI search! In this notebook, we aren't just matching keywords; we are building a system that **sees**. By using the `multi2vec-clip` module, we can project both images and text into the same high-dimensional vector space.

### The Goal:
1. **Initialize** a Weaviate instance that understands cross-modal relationships.
2. **Ingest** raw image data converted to Base64.
3. **Search** for images using only a text prompt (Text-to-Image).
4. **Search** for images using another image (Image-to-Image similarity).

In [ ]:
import pickle, weaviate, json, os, IPython, base64

### We first need to start a local instance of Weaviate using Docker.

1. This can be done by opening up a terminal in the folder with the provided `docker-compose.yml` file in it and typing:
```bash
docker compose up
```

2. Later, in order to bring Weaviate down you can just go into this terminal window and type `Ctrl + C`

In [ ]:
# Connect to locally hosted Weaviate
client = weaviate.Client("http://localhost:8080")

# Check if the service is live
print("Is Weaviate ready?", client.is_ready())

### Q1: Specify an ML model that can understand both images and text into Weaviate

### 🧠 The CLIP Model
To make our database "multi-modal," we use **CLIP (Contrastive Language-Image Pre-training)**. It was trained on millions of image-caption pairs to learn how a picture of a "golden retriever" relates to the written words "golden retriever."



In [ ]:
# Delete existing schema if it exists to start fresh
if client.schema.exists("Meme"):
    client.schema.delete_class("Meme")

class_obj = {
    "class": "Meme",
    "description": "A class to store images and their vectors",
    "moduleConfig": {
        "multi2vec-clip": {
            "imageFields": ["image"]
        }
    ExternalDependencies: [],
    "vectorizer": "multi2vec-clip",
    "properties": [
        {"name": "image", "dataType": ["blob"]},
        {"name": "filename", "dataType": ["string"]}
    ]
}

client.schema.create_class(class_obj)
print("Schema class 'Meme' created with CLIP vectorizer")

### Q2: Add the provided data to your vector database - feel free to use your own images for this aswell!

### 📦 Storing Blobs
Weaviate stores images as **Base64 encoded strings**. During import, Weaviate automatically intercepts these strings, passes them through the CLIP model, and stores the resulting vector alongside the image data.

In [ ]:
with client.batch(batch_size=10) as batch:
    for img_file in os.listdir("Images/"):
        if img_file.endswith((".jpg", ".jpeg", ".png", ".JPEG")):
            with open(f"Images/{img_file}", "rb") as f:
                b64_image = base64.b64encode(f.read()).decode('utf-8')
            
            properties = {
                "image": b64_image,
                "filename": img_file
            }
            batch.add_data_object(properties, "Meme")
    
print("All images added and vectorized!")

### Q3. Search the data with a text query

### 🔍 Text-to-Image Search
Here, we provide a string like "a sunset over the mountains." Weaviate turns that string into a vector and finds the images whose vectors are mathematically closest (using Cosine Similarity).



In [ ]:
text_query = "ocean waves"

res = (
    client.query
    .get("Meme", ["filename", "image"])
    .with_near_text({"concepts": [text_query]})
    .with_limit(3)
    .do()
)

print(f"Found {len(res['data']['Get']['Meme'])} images matching '{text_query}'")

**Visualize some of the returned images:**

In [ ]:
# Display result 1
if res['data']['Get']['Meme']:
    img_data = res['data']['Get']['Meme'][0]['image']
    IPython.display.display(IPython.display.Image(data=base64.b64decode(img_data), width=300))

In [ ]:
# Display result 2
if len(res['data']['Get']['Meme']) > 1:
    img_data = res['data']['Get']['Meme'][1]['image']
    IPython.display.display(IPython.display.Image(data=base64.b64decode(img_data), width=300))

In [ ]:
# Display result 3
if len(res['data']['Get']['Meme']) > 2:
    img_data = res['data']['Get']['Meme'][2]['image']
    IPython.display.display(IPython.display.Image(data=base64.b64decode(img_data), width=300))

### Feel free to perform more searches and see if you can explain why specific text search queries return images! 

#### You've successfully used a vector database to build text-to-image search!

### Q4. Search the data with an image query.

- **Here we will pass in images that are not in the vector database and search for the most similar images as determined by vector search** 

#### First visualize the image you want to query the database with:

In [ ]:
test_img_path = 'TestImages/n03417042_28762.JPEG'
IPython.display.Image(filename=test_img_path, width=300) 

#### Now let's write a query to search using this image:

In [ ]:
with open(test_img_path, "rb") as f:
    test_b64 = base64.b64encode(f.read()).decode('utf-8')

imres = (
    client.query
    .get("Meme", ["filename", "image"])
    .with_near_image({"image": test_b64})
    .with_limit(3)
    .do()
)

print("Image-to-Image search completed.")

**Visualize the images most similar to the input image**

In [ ]:
if imres['data']['Get']['Meme']:
    IPython.display.display(IPython.display.Image(data=base64.b64decode(imres['data']['Get']['Meme'][0]['image']), width=300))

In [ ]:
if len(imres['data']['Get']['Meme']) > 1:
    IPython.display.display(IPython.display.Image(data=base64.b64decode(imres['data']['Get']['Meme'][1]['image']), width=300))

In [ ]:
if len(imres['data']['Get']['Meme']) > 2:
    IPython.display.display(IPython.display.Image(data=base64.b64decode(imres['data']['Get']['Meme'][2]['image']), width=300))